In [1]:
#x01_populationStatistics

In [2]:
from chewc.chewc import *

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class MyNetwork(nn.Module):
    def __init__(self, L, P):
        super(MyNetwork, self).__init__()
        
        # Layers for input1 and input3
        self.fc1 = nn.Linear(L, 128)
        self.fc2 = nn.Linear(128, 64)
        
        # Layers for input2 and input4
        self.fc3 = nn.Linear(P, 128)
        self.fc4 = nn.Linear(128, 64)
        
        # Layers for the upper triangle of input_matrix
        self.fc_matrix = nn.Linear(L * (L + 1) // 2, 256)
        
        # Final fully connected layers
        self.fc_final1 = nn.Linear(64 * 4 + 256, 64)
        self.fc_final2 = nn.Linear(64, 2)
    
    def forward(self, input1, input2, input3, input4, input_matrix):
        
        # Process input1 and input3
        x1 = F.relu(self.fc1(input1))
        x1 = F.relu(self.fc2(x1))
        
        x3 = F.relu(self.fc1(input3))
        x3 = F.relu(self.fc2(x3))
        
        # Process input2 and input4
        x2 = F.relu(self.fc3(input2))
        x2 = F.relu(self.fc4(x2))
        
        x4 = F.relu(self.fc3(input4))
        x4 = F.relu(self.fc4(x4))
        
        # Process input_matrix
        indices = torch.triu_indices(input_matrix.size(1), input_matrix.size(2))
        upper_triangle = input_matrix[:, indices[0], indices[1]]
        x_matrix = F.relu(self.fc_matrix(upper_triangle))
        
        # Concatenate all features
        x = torch.cat((x1, x2, x3, x4, x_matrix), dim=1)
        
        # Final fully connected layers
        x = F.relu(self.fc_final1(x))
        output = self.fc_final2(x)
        
        return output
    
# Example usage
# batch_size = 1
# L = 1000  # Updated to match the size of the given tensor
# P = 200
# model = MyNetwork(L, P)
# input1 = torch.randn((batch_size, L))
# input2 = torch.randn((batch_size, P))
# input3 = torch.randn((batch_size, L))
# input4 = torch.randn((batch_size, P))
# input_matrix = torch.tensor([[[ 1.0000,  0.0256,  0.1095,  0.0666, -0.0401],
#                               [ 0.0256,  1.0000,  0.1458,  0.0527, -0.0487],
#                               [ 0.1095,  0.1458,  1.0000,  0.0550, -0.0285],
#                               [ 0.0666,  0.0527,  0.0550,  1.0000,  0.0461],
#                               [-0.0401, -0.0487, -0.0285,  0.0461,  1.0000]]])
# input_scalar = torch.randn((batch_size,))
# # Forward pass
# output = model(input1, input2, input3, input4, input_matrix, input_scalar)
# print(output)


In [21]:
n_chr = 1
n_loci = 1000
founder_pop_size = 200

G = Genome(n_chr, n_loci)
founder_pop = create_pop(G, create_random_pop(G, founder_pop_size))
T = Trait(G, founder_pop, target_mean=0.0, target_variance=1.0)

sim = BreedingSimulation(G, T, h2
                         =0.2, reps=2, pop_size=founder_pop_size, selection_fraction=0.5)

pop_stat_in  = sim.population.haplotypes.sum(dim=1).view((sim.population.size, n_chr* n_loci)).float()
pop_stat = population_statistics(pop_stat_in)

# Example usage
batch_size = 1
L = n_loci  # Updated to match the size of the given tensor
P = founder_pop_size
model = MyNetwork(L, P)


In [24]:
n_chr = 1
n_loci = 1000
founder_pop_size = 200

G = Genome(n_chr, n_loci)
founder_pop = create_pop(G, create_random_pop(G, founder_pop_size))
T = Trait(G, founder_pop, target_mean=0.0, target_variance=1.0)

sim = BreedingSimulation(G, T, h2
                         =0.2, reps=2, pop_size=founder_pop_size, selection_fraction=0.5)

pop_stat_in  = sim.population.haplotypes.sum(dim=1).view((sim.population.size, n_chr* n_loci)).float()
pop_stat = population_statistics(pop_stat_in)

# Example usage
batch_size = 1
L = n_loci  # Updated to match the size of the given tensor
P = founder_pop_size
model = MyNetwork(L, P)
input1 = pop_stat['maf'].unsqueeze(0)
input2 = pop_stat['inbreeding_coefficient'].unsqueeze(0) 
input3 = pop_stat['heterozygosity'].unsqueeze(0)
input4 = pop_stat['genotype_frequencies'][1].unsqueeze(0)
input_matrix = pop_stat['ld_matrix'].unsqueeze(0)
input_scalar = torch.tensor(2.).unsqueeze(0)
output = model(input1, input2, input3, input4, input_matrix)
print(output)

tensor([[ 0.0960, -0.1597]], grad_fn=<AddmmBackward0>)


tensor([[9235.8350, 5457.0605]], grad_fn=<AddmmBackward0>)


In [7]:
pop_stat['maf'].unsqueeze(0).shape

torch.Size([1, 1000])

In [8]:
pop_stat['maf'].shape

torch.Size([1000])

In [10]:
pop_stat['allele_frequencies'].shape


torch.Size([1000])

In [11]:
pop_stat['genotype_frequencies'][0].shape


torch.Size([200, 1000])

In [12]:
pop_stat['genotype_frequencies'][1].shape


torch.Size([200])

In [13]:
pop_stat['heterozygosity'].shape


torch.Size([1000])

In [14]:
pop_stat['effective_population_size']

tensor(168549.9688)